In [362]:
import pandas as pd 
import neattext as nt 
data = pd.read_excel('raw_data/IVM6311_Testing_scripts.xlsx',sheet_name='Procedure')
# data.head()
# for col in data.columns:
#     data[col][0] = data[col][0].split('\n')
data.to_csv('raw_data/Procedure_data.csv')
data

,Run_startup,Enable_Ana_Testpoint,DACPA_tourn_on_wo_calibration,Procedura_Maurizio,Boost_Test_Default,DAC-PA TDM_768_IN,DAC-PA TDM_768_IN_and_Path_generator,DAC-PA PDM_3MHz_IN,DAC-PA PDM_6MHz_IN,DAC-PA PDM_12MHz_IN,DAC-PA ANALOG_IN_DC_COUPLED,DAC-PA ANALOG_IN_AC_COUPLED,BOOST_FORCE_VBSO19V
0,Force__VBAT__3.6V\nForce__VDDIO__1.8V\nForce__...,"0x19_0x00 ""Enable reference analog test""\nForc...","0x13[4]_1b1 ""Disable authomatic calibration at...",Procedura per consentire la misura dell'offset...,Force__VBIAS__5V\nForce__VBSO__3.6V\nForce__SW...,"Run_Startup\n0xFE_0x00 ""in base page""\n0x13[4]...","Run_Startup\n0xFE_0x00 ""in base page""\n0x13[4]...","Run_Startup\n0xFE_0x00 ""in base page""\n0x13[4]...","Run_Startup\n0xFE_0x00 ""in base page""\n0x13[4]...","Run_Startup\n0xFE_0x00 ""in base page""\n0x13[4]...","Run_Startup\n0xFE_0x00 ""in base page""\n0x13[4]...","Run_Startup\n0xFE_0x00 ""in base page""\n0x13[4]...",Force__VBIAS__19V\nForce__VBSO__19V\n\n0xFE_0x...


### 1. List the procedure names 
<p style = ' color: red' > ! Do not ever change the name of the procedures </p>

In [232]:
procuedure_names = data.columns.to_list()
print(procuedure_names)

['Run_startup', 'DACPA_tourn _on_wo_calibration', 'Procedura_Maurizio', 'Boost_Test_Default', 'DAC-PA TDM_768_IN', 'DAC-PA TDM_768_IN_and_Path_generator', 'DAC-PA PDM_3MHz_IN', 'DAC-PA PDM_6MHz_IN', 'DAC-PA PDM_12MHz_IN', 'DAC-PA ANALOG_IN_DC_COUPLED', 'DAC-PA ANALOG_IN_AC_COUPLED', 'BOOST_FORCE_VBSO19V']


In [14]:
# sample of the procedure 
data['Boost_Test_Default'].apply(print)

['Force__VBIAS__5V', 'Force__VBSO__3.6V', '', '0xFE_0x00 "Select page 0"', '0xB0_0x00 "en 1 bst low"', '0xB1_0x00 "en 2 bst low"', '0xB3[4:3]_0x00 "en 3 bst low"', '0x18[6]_0x01 "Enable Mirr"', '0xB1[6]_0x01 "Bootstrap en"', '0xB2[1]_0x00 "ls_sel off"', '0xB4[2:0]_0x00 "hsa_hsb_byp off"', '0xFE_0x01 "Select page 1"', '0x17[6]_0x01 "Force Mirr"', '0x16[6]_0x01 "Force Bootstrap"']


0    None
Name: Boost_Test_Default, dtype: object

In [201]:
import re 
float(*re.findall('[0-9\.\-]+', 'Force_Current__VBSO__-400mA'))

-400.0

<h3 style='color: #40E0D0'> Extract the Force signal</p>

<p style='color:red'> check all type of force signals </p>

In [457]:
from typing import List
import json
import random
import re 
force_instructions = {}
for sheet in ['reference','Boost', 'Digital']:
    sheet = 'raw_data/'+sheet+'_data.csv'
    data = pd.read_csv(sheet)
    for col in data.columns.to_list()[1:]:
        instructions = data[col][1].split('\n')
        for i,instruction in enumerate(instructions):
            if re.match('force',instruction.lower()) and '0x' not in instruction.lower():
                force_instructions.update({random.randint(0,1000) : instruction})

data = pd.read_csv('raw_data/Procedure_data.csv')
for col in data.columns.to_list()[1:]:
    for procedure in data[col]:
        for instruction in procedure.split('\n'):
            
            if 'Force' in instruction and '0x' not in instruction:
                force_instructions.update({random.randint(1000,2000) : instruction})
                
for i in force_instructions.values():          
    print(i)
    
with open('raw_data/force_instruction_types.json', 'w') as file:
    json.dump({"":force_instructions}, file)
    

Force__VBAT__3.6V
Force__SDWN__1.8V
Force__VBIAS__5V
Force__VBSO__3.6V
Force__SW__3.6V
Force__VBIAS__5V
Force__VBSO__3.6V
Force_Current__SW__400mA "chiedere comando corrente"
Force__VBAT__5V
Force__VDDIO__1.8V
Force__VDD__1.8V
Force__VBIAS__5V
Force__VBSO__5V
Force__SDWN__1.8V
Force_Current__SW__-400mA "sink current -400mA"
Force__VBIAS__5V
Force__SW__3.6V
Force_Current__VBSO__-400mA "sink"
Force__VBIAS__12V
Force__VBSO__11V
Force__Current__SW__100mA "inject"
Force__VBIAS__5V
Force__VBSO__3.6V
Force__Current__SW__-100mA "sink"
Force__VBIAS__5V
Force__extsignal "with a voltage lower than vbat (also 0 should be fine) so that the boost remains in bypass"
Force__extsignal "with a voltage lower than vbat (also 0 should be fine) so that the boost remains in bypass"
Force__VBAT__5V
Force__VDDIO__1.8V
Force__VDD__1.8V
Force__VBIAS__5V
Force__VBSO__5V
Force__SW__5V
Force__Ramp "da 4.5V con step di 50mV in discesa finchè non commuta 3-4 volte per capire la threshold"
Force__VPAP_OUT__ 0V "PA pos

In [458]:
import re 
from typing import List
def extract_Force__Instruction(Instruction: str):
    force_signal = {}
    def delist(x:List):
        if isinstance(x,List) and len(x) != 0 :
            x=x[-1]
        elif len(x) == 0:
            x = 0
        else:
            x = x
        return x 
    
    def value_unit(value, unit):
        if 'm' in unit:
            value = value*10**-3
            unit = unit.strip('m').capitalize()
            
        elif 'u' in unit:
            value = value*10**-6
            unit = unit.strip('u').capitalize()
            
        elif 'n' in unit:
            value = value*10**-9
            unit = unit.strip('n').capitalize()
            
        elif 'k' in unit:
            value = value*10**3
            unit = unit.strip('k').capitalize()
            
        else :
            value = value
            unit = unit.capitalize()

        return value, unit
    
    # match the small or capital letter force key word
    if re.match('[Force]|[force]', Instruction) and re.search('__', Instruction) :
        signal = Instruction
        # signal = re.findall(re.compile('([A-Za-z0-9\.\-]+)'), Instruction)[1:] # find the force signal and with the value 
        #remove the comments if there are any comments 
        if re.search(r"\"(.*?)\"",signal):
            signal = re.sub(r"\"(.*?)\"",'',signal)

        signal = signal.split('__')[1:] # find the force signal and with the value 
        #check the signal and value, array length 
        # if the instruction has signal and the value array length must the 2 
        signal_length = len(signal)
        if signal_length == 2:
            ########################
            # for force instruction length 2 
            # @pattren 'Force_Current__SW__400mA'
            ########################

            signal_name = signal[0]
                #check the signal unit and extract the number 
            unit = delist(re.findall('[A-Za-z]+', signal[1].lower()))
            if  value := float(delist(re.findall('[0-9\.\-]+', signal[1]))) :
                # check the volatage or current 
                value, unit = value_unit(value,unit)
            # else check for the open or close 
                
            elif value := delist(re.findall('OPEN', signal[1])):
                unit = None 
            elif value := delist(re.findall('CLOSE', signal[1])):
                    unit = None
                    
            force_signal = {
            'Signal' : signal_name,
            'Value'  : value ,
            'Unit'   : unit
            }
        
        elif signal_length == 3:
            ########################
            # for force instruction length 2 
            # @pattren 'Force_Current__SW__400mA'
            ########################
            signal_name = signal[1]
            #check the signal unit and extract the number 
            unit = delist(re.findall('[A-Za-z]+', signal[2].lower()))
            if  value := float(delist(re.findall('[0-9\.\-]+', signal[2]))) :
                # check the volatage or current 
                value, unit = value_unit(value,unit)

            # else check for the open or close 
            elif value := delist(re.findall('OPEN', signal[1])):
                unit = None 
            elif value := delist(re.findall('CLOSE', signal[1])):
                unit = None 

            force_signal = {
                'Signal' : signal_name,
                'Value'  : value ,
                'Unit'   : unit
            }
        
        
    return force_signal    
# extract_Force__Instruction(Instruction = 'Force_Current__SW__400mA')
# extract_Force__Instruction(Instruction = 'Force_Current__VBSO__-400mA')
# extract_Force__Instruction(Instruction = 'Force_Current__SW__400mA "chiedere comando corrente"')
extract_Force__Instruction(Instruction = 'Force__SDWN__OPEN')
# extract_Force__Instruction(Instruction = 'Force__SDWN__1.8V')

{'Signal': 'SDWN', 'Value': 'OPEN', 'Unit': None}

<h5 style="color : #ffbd33" > Test all types of Force signal in the document </h5>

In [459]:
for instruction in force_instructions.values():
    print(instruction)
    if force_instruction_parse := extract_Force__Instruction(instruction):
        print(force_instruction_parse)
    else:
        print('.........', instruction)

Force__VBAT__3.6V
{'Signal': 'VBAT', 'Value': 3.6, 'Unit': 'V'}
Force__SDWN__1.8V
{'Signal': 'SDWN', 'Value': 1.8, 'Unit': 'V'}
Force__VBIAS__5V
{'Signal': 'VBIAS', 'Value': 5.0, 'Unit': 'V'}
Force__VBSO__3.6V
{'Signal': 'VBSO', 'Value': 3.6, 'Unit': 'V'}
Force__SW__3.6V
{'Signal': 'SW', 'Value': 3.6, 'Unit': 'V'}
Force__VBIAS__5V
{'Signal': 'VBIAS', 'Value': 5.0, 'Unit': 'V'}
Force__VBSO__3.6V
{'Signal': 'VBSO', 'Value': 3.6, 'Unit': 'V'}
Force_Current__SW__400mA "chiedere comando corrente"
{'Signal': 'SW', 'Value': 0.4, 'Unit': 'A'}
Force__VBAT__5V
{'Signal': 'VBAT', 'Value': 5.0, 'Unit': 'V'}
Force__VDDIO__1.8V
{'Signal': 'VDDIO', 'Value': 1.8, 'Unit': 'V'}
Force__VDD__1.8V
{'Signal': 'VDD', 'Value': 1.8, 'Unit': 'V'}
Force__VBIAS__5V
{'Signal': 'VBIAS', 'Value': 5.0, 'Unit': 'V'}
Force__VBSO__5V
{'Signal': 'VBSO', 'Value': 5.0, 'Unit': 'V'}
Force__SDWN__1.8V
{'Signal': 'SDWN', 'Value': 1.8, 'Unit': 'V'}
Force_Current__SW__-400mA "sink current -400mA"
{'Signal': 'SW', 'Value': -0.4,

<h3 style='color: #2ecc71' > Extract Measure signal </h3>

<h5 style='color: #ff5733'> Extract all the  measure signal pattrens</h5>

In [468]:
from typing import List
import json
import random
import re 
measure_instructions = {}
for sheet in ['Digital','Reference','Boost','DAC+PA','AZ_COMP']:
    sheet = 'raw_data/'+sheet+'_data.csv'
    data = pd.read_csv(sheet)
    for col in data.columns.to_list()[1:]:
        instructions = data[col][1].split('\n')
        for i,instruction in enumerate(instructions):
            if re.match('meas',instruction.lower()) and '0x' not in instruction.lower():
                measure_instructions.update({random.randint(0,1000) : instruction})
print(measure_instructions)

{740: 'Measure__Voltage__SDWN', 389: 'Measure__Voltage__SDWN', 489: 'Measure__Voltage__SDWN', 298: 'Measure__Voltage__SDWN', 391: 'Measure__Frequency__FSYN', 313: 'Measure__Current__SDWN "tswitch mirr measure"', 443: 'Measure__Voltage__FSYN "as soon as trigger, need command"', 236: 'Measure__Voltage__SDWN', 939: 'Measure__Voltage__SDWN', 447: 'Measure__Voltage__VBSO', 474: 'Measure__Voltage__FSYN "it should be 0"', 272: 'Measure__THD__AP', 792: 'Measure__Voltage__FSYN', 543: 'Measure__Voltage__SDI "FSYN should be 0 and SDI 1"', 247: 'Measure__Voltage__FSYN', 110: 'Measure__current__VDDIO', 29: 'Measure__Voltage__FSYN', 178: 'Measure__Voltage_SDI "da and a, FSYN should be 0 and SDI 0"', 982: 'Measure__Voltage__FSYN "out_comp, it should be 0"', 487: 'Measure__Voltage__FSYN "out_comp, it should be 1"', 85: 'Measure__Voltage__FSYN "da and a, it should be 0"', 378: 'Measure__Voltage__SDI "FSYN should remain 0 since vbias is not enabled and SDI 1"', 483: 'Measure__Voltage__FSYN', 283: 'Measu

<h5 style="color : #ffbd33" > Parse Measure signal </h5>

In [469]:

import re 
from typing import List
def extract_Measure__Instruction(Instruction: str):
    measure_signal = {}
    signal = Instruction

    if re.search(r"\"(.*?)\"",signal):
        signal = re.sub(r"\"(.*?)\"",'',signal)
        
    if re.match('meas', signal.lower()):
        signal = re.findall('[A-Za-z0-9\.]+', signal)[1:] # find the measure signal and with the value 
        #check the signal and value, array length 
        # if the instruction has signal and the value array length must the 2 
        if len(signal) >= 2:
            signal_name = signal[1]
            unit = signal[0]

            measure_signal = {
                'Signal' : signal_name,
                'Unit'   : unit
            }

    return measure_signal    
extract_Measure__Instruction(Instruction = 'Measure__Current__SDWN')

{'Signal': 'SDWN', 'Unit': 'Current'}

<h5 style="color :#33c5ff" > Testing the Measure signal </h5>

In [470]:
for instruction in measure_instructions.values():
    print(instruction)
    if measure_instruction_parse := extract_Measure__Instruction(instruction):
        print(measure_instruction_parse)
    else:
        print('.........', instruction)

Measure__Voltage__SDWN
{'Signal': 'SDWN', 'Unit': 'Voltage'}
Measure__Voltage__SDWN
{'Signal': 'SDWN', 'Unit': 'Voltage'}
Measure__Voltage__SDWN
{'Signal': 'SDWN', 'Unit': 'Voltage'}
Measure__Voltage__SDWN
{'Signal': 'SDWN', 'Unit': 'Voltage'}
Measure__Frequency__FSYN
{'Signal': 'FSYN', 'Unit': 'Frequency'}
Measure__Current__SDWN "tswitch mirr measure"
{'Signal': 'SDWN', 'Unit': 'Current'}
Measure__Voltage__FSYN "as soon as trigger, need command"
{'Signal': 'FSYN', 'Unit': 'Voltage'}
Measure__Voltage__SDWN
{'Signal': 'SDWN', 'Unit': 'Voltage'}
Measure__Voltage__SDWN
{'Signal': 'SDWN', 'Unit': 'Voltage'}
Measure__Voltage__VBSO
{'Signal': 'VBSO', 'Unit': 'Voltage'}
Measure__Voltage__FSYN "it should be 0"
{'Signal': 'FSYN', 'Unit': 'Voltage'}
Measure__THD__AP
{'Signal': 'AP', 'Unit': 'THD'}
Measure__Voltage__FSYN
{'Signal': 'FSYN', 'Unit': 'Voltage'}
Measure__Voltage__SDI "FSYN should be 0 and SDI 1"
{'Signal': 'SDI', 'Unit': 'Voltage'}
Measure__Voltage__FSYN
{'Signal': 'FSYN', 'Unit': 'V

In [108]:
import re 
from typing import List
def extract_Delay__Instruction(Instruction: str):
    delay = 0
    # meath the measure signal
    Instruction = Instruction.lower()
    if re.match('wait', Instruction):
        signal = re.findall('[A-Za-z0-9\.]+', Instruction)[1:] # find the measure signal and with the value 
        #check the signal and value, array length 
        # if the instruction has signal and the value array length must the 2 
        if len(signal) >= 2:
            print()
            delay = float( (lambda x :  float(*x) if isinstance(x,List) else float(*x))(re.findall('[0-9\.]+', signal[1])))
            # check for the milli seconds 
            if 'ms' in signal[1]:
                delay  = delay* 10**-3
            # check for the micro seconds 
            elif 'us' in signal[1]:
                delay  = delay* 10**-6
    return delay    
extract_Delay__Instruction(Instruction = 'Wait__delay__0.1ms')

0.0001